In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

In [2]:
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%config SqlMagic.displaylimit = 100

In [3]:
%%sql
SELECT name, type
  FROM sqlite_master
 WHERE type='table';

name,type
sqlite_sequence,table
facts,table


### Introduction

The data in this project derived from https://www.cia.gov/the-world-factbook/ which contains statistics about all of countries in the world. The factbook contains demographic information such as population, population growth and total area of water and land. The table contains an outlier row that present total number of world population instead total population per country, therefore in the further analysis, this row will be excluded so the data correctly present total population per country. Below are some interesting questions pertaining world statistics. 

1. Minimum and maximum population number and population growth

In [4]:
%%sql

SELECT MIN (population) AS Minimum_population,
       MAX (population) AS Maxmimum_population,
       MIN (population_growth) AS Minimum_population_growth,
       MAX (population_growth) AS Maximum_population_growth
  FROM facts
  WHERE name <> 'World';

Minimum_population,Maxmimum_population,Minimum_population_growth,Maximum_population_growth
0,1367485388,0.0,4.02


Note: Antartica has 0 population since there is no indigenous inhabitants, but there are both permanent and summer-only staffed research stations. 

2. World average population and area

In [5]:
%%sql 

SELECT AVG(population) AS average_population, 
       AVG(area) AS average_area
  FROM facts
  WHERE name <> 'World';

average_population,average_area
32242666.56846473,555093.546184739


3. List of densely populated countries

In [6]:
%%sql 

SELECT name AS country_name, 
       population AS country_population,
       ROUND((SELECT AVG(population)
         FROM facts
         WHERE name <>'World'),2) AS avg_world_population,
       area AS country_area, 
       ROUND((SELECT AVG(area)
         FROM facts
         WHERE name <>'World'),2) AS avg_area_population
  FROM facts
 WHERE name <> 'World' AND population>(SELECT AVG(population)
                                           FROM facts
                                        WHERE name<>'World')
    
    AND area<(SELECT AVG(area)
               FROM facts
               WHERE name<>'World')

country_name,country_population,avg_world_population,country_area,avg_area_population
Bangladesh,168957745,32242666.57,148460,555093.55
Germany,80854408,32242666.57,357022,555093.55
Iraq,37056169,32242666.57,438317,555093.55
Italy,61855120,32242666.57,301340,555093.55
Japan,126919659,32242666.57,377915,555093.55
"Korea, South",49115196,32242666.57,99720,555093.55
Morocco,33322699,32242666.57,446550,555093.55
Philippines,100998376,32242666.57,300000,555093.55
Poland,38562189,32242666.57,312685,555093.55
Spain,48146134,32242666.57,505370,555093.55


4. Country with the highest number of population 

In [7]:
%%sql 
SELECT name,
       MAX(population)country_with_highest_population
  FROM facts
  WHERE name <> 'World'
  LIMIT 10;


name,country_with_highest_population
China,1367485388


5. Country with the highest population growth rate

In [8]:
%%sql 

SELECT name,
       MAX(population_growth)country_with_highest_population_growth
  FROM facts
  WHERE name <> 'World'
  LIMIT 10;

name,country_with_highest_population_growth
South Sudan,4.02


6. Countries that have the highest ratios of water to land

In [9]:
%%sql 

SELECT name,area_water/area_land AS ratio_water_to_land
  FROM facts
  WHERE name <> 'World'
  GROUP BY name
  ORDER BY ratio_water_to_land DESC
 LIMIT 10;

name,ratio_water_to_land
British Indian Ocean Territory,905
Virgin Islands,4
Zimbabwe,0
Zambia,0
Yemen,0
Western Sahara,0
West Bank,0
Wallis and Futuna,0
Wake Island,0
Vietnam,0


 7. Countries that have more water than land

In [10]:
%%sql 

SELECT name,area_water,area_land 
  FROM facts
  WHERE name <> 'World' AND area_water>area_land
 LIMIT 10;

name,area_water,area_land
British Indian Ocean Territory,54340,60
Virgin Islands,1564,346


8. Countries that will add the most people to their populations next year 

In [11]:
%%sql 

SELECT name, population_growth
  FROM facts
 WHERE name <>'World' and population_growth <> 'None'
 ORDER BY population_growth DESC
 LIMIT 10;

name,population_growth
South Sudan,4.02
Malawi,3.32
Burundi,3.28
Niger,3.25
Uganda,3.24
Qatar,3.07
Burkina Faso,3.03
Mali,2.98
Cook Islands,2.95
Iraq,2.93


 9. Countries that have a higher death rate than birth rate

In [12]:
%%sql 

SELECT name, population, population_growth, birth_rate,death_rate
  FROM facts
 WHERE name <> 'World' AND death_rate > birth_rate
 LIMIT 10;

name,population,population_growth,birth_rate,death_rate
Austria,8665550,0.55,9.41,9.42
Belarus,9589689,0.2,10.7,13.36
Bosnia and Herzegovina,3867055,0.13,8.87,9.75
Bulgaria,7186893,0.58,8.92,14.44
Croatia,4464844,0.13,9.45,12.18
Czech Republic,10644842,0.16,9.63,10.34
Estonia,1265420,0.55,10.51,12.4
Germany,80854408,0.17,8.47,11.42
Greece,10775643,0.01,8.66,11.09
Hungary,9897541,0.22,9.16,12.73


10. Countries have the highest population/area ratio

In [13]:
%%sql 
SELECT name, population, area, population/area AS ratio_population_area
  FROM facts
 WHERE name <> 'World' AND area <>'None' AND population <>'None' AND area <>0
 ORDER BY ratio_population_area DESC
 LIMIT 10;

name,population,area,ratio_population_area
Macau,592731,28,21168
Monaco,30535,2,15267
Singapore,5674472,697,8141
Hong Kong,7141106,1108,6445
Gaza Strip,1869055,360,5191
Gibraltar,29258,6,4876
Bahrain,1346613,760,1771
Maldives,393253,298,1319
Malta,413965,316,1310
Bermuda,70196,54,1299
